In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time

2023-05-06 18:53:46.114406: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:53:46.406934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:53:46.408344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 18:53:48.068931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("./dataset/inspiration.csv", index_col = 0)
new_data = data[['Category', 'Quote']]
# new_data

In [3]:
new_data['Category'].unique()

array(['LOVE', 'LISTENING', 'STEWARDSHIP', 'RESILIENCE', 'EXPLORING',
       'KINDNESS', 'HELPING OTHERS', 'OPTIMISM', 'GIVING',
       'COMMON GROUND', 'HOPE', 'ACHIEVEMENT', 'GRATITUDE', 'CHARITY',
       'PRACTICE', 'GET OUTSIDE', 'FORGIVENESS', 'CHARACTER', 'LAUGHTER',
       'HUMILITY', 'ACCEPTANCE', 'DEDICATION', 'BELIEVE IN YOURSELF',
       'SMILE', 'PEACE', 'PERSPECTIVE', 'LIVE YOUR DREAMS', 'CONFIDENCE',
       'INNOVATION', 'LEARNING', 'UNITY', 'DISCOVERY', 'SELFLESSNESS',
       'FITNESS', 'LEADERSHIP', 'FRIENDSHIP', 'CURIOSITY',
       'ENCOURAGEMENT', 'SPREAD YOUR WINGS', 'HARD WORK', 'LIVE LIFE',
       'JOY', 'WISDOM', 'PERSISTENCE', 'INSPIRATION', 'OVERCOMING',
       'CREATIVITY', 'FAMILY', 'CHEER', 'GIVING BACK', 'COMPASSION',
       'AMBITION', 'LOYALTY', 'GRIT', 'EDUCATION', 'APPRECIATING NATURE',
       'INTEGRITY', 'EMPATHY', 'SELF-CARE', 'TEAMWORK',
       'INCLUDING OTHERS', 'COURAGE', 'PERSEVERANCE', 'APPRECIATION',
       'MOTIVATION', 'MENTORING', 'HUMOR', '

In [4]:
file1 = open("MyFile.txt", "w")
the_length = new_data.shape[0]
for i in range(0,the_length):
    file1.write(new_data["Category"][i])
    file1.write(": ")
    file1.write(new_data["Quote"][i])
    file1.write("\n")
    # file1.write("\n")
file1.close()

In [8]:
from keras.callbacks import LambdaCallback
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file
import string
import io

# print('\nPreparing the sentences...')
# max_sentence_len = 40
# with open("MyFile.txt") as file_:
#   docs = file_.readlines()
# sentences = [[word for word in doc.lower().translate(string.punctuation).split()] for doc in docs]
# # print(sentences)
# print('Num sentences:', len(sentences))
corpus = "MyFile.txt"
with io.open(corpus) as f:
    text = f.read().lower().replace('\n', ' \n ')
print('Corpus length in characters:', len(text))

text_in_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print('Corpus length in words:', len(text_in_words))

Corpus length in characters: 219304
Corpus length in words: 39583


In [9]:
# Calculate word frequency
MIN_WORD_FREQUENCY = 1
word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))
print(words)

Unique words before ignoring: 5570
Ignoring words with frequency < 1
Unique words after ignoring: 5570
['\n', '"am', '"i', '"timing"', '"what', '"what!', '&', "'i", "'it", "'listen'", "'no'", "'silent'.", "'things", "'til", "'till", "'who", "'why", "'yes'", "'you", '(piglet)', '(which', '(winnie-the-pooh)', '-', '---', '...', '1%', '10', '10,000', '211', '212', '26', '300', '9,000', '99%', 'a', 'abandon', 'abandon.', 'abandoned.', 'abates', 'abilities.', 'ability', 'ability\u2028to', 'able', 'about', 'about,', 'about.', 'above', 'above:', 'absence', 'absolute', 'absolutely', 'absurd', 'abundance', 'abundance.', 'abundant', 'academics', 'accept', 'accept,', 'acceptance', 'acceptance,', 'acceptance.', 'acceptance:', 'accepted', 'accepting', 'accepts', 'accessible', 'accident.', 'accomplish', 'accomplish.', 'accomplished', 'accomplished.', 'accomplished…', 'accomplishment', 'accomplishment.', 'accomplishments', 'accumulated', 'accurately', 'achieve', 'achieved', 'achieved.', 'achievement'

In [11]:
# cut the text in semi-redundant sequences of SEQUENCE_LEN words
STEP = 1
SEQUENCE_LEN = 60
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 0
Remaining sequences: 39523


In [13]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)
(sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 38732
Size of test set = 791


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
model = Sequential()
dropout = 0.2
model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
if dropout > 0:
    model.add(Dropout(dropout))
model.add(Dense(len(words)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

2023-05-06 19:11:34.185326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-06 19:11:34.191656: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-06 19:11:34.198038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [20]:
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y
file_path = "./LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-accuracy{accuracy:.4f}-val_loss{val_loss:.4f}-val_accuracy{val_accuracy:.4f}" % (
    len(words),
    SEQUENCE_LEN,
    MIN_WORD_FREQUENCY
)
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    # examples_file = open("examples_file.txt", "w")
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]
examples_file = open("examples.txt.", "w")
BATCH_SIZE = 128
model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=5,
                        callbacks=callbacks_list,
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

/tmp/ipykernel_378/3385089504.py:68: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
/tmp/ipykernel_378/3385089504.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
/tmp/ipykernel_378/3385089504.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` he

Epoch 1/100


2023-05-06 19:11:38.164980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-06 19:11:38.172484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-06 19:11:38.177820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

108/303 [=========>....................] - ETA: 4:57 - loss: 7.0208 - accuracy: 0.0447

KeyboardInterrupt: 